# Text Classification with BERT (ASAP Dataset)


In [57]:
import numpy as np
import pandas as pd
import os
import string
import re
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pickle


import warnings
warnings.filterwarnings("ignore")

In [58]:
from google.colab import files
uploaded = files.upload()

Saving training_set_rel3.tsv to training_set_rel3 (1).tsv


In [59]:
import io
data = io.BytesIO(uploaded['training_set_rel3.tsv'])

In [60]:
df = pd.read_csv(data , sep='\t', encoding='ISO-8859-1')
df.head()

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
0,1,1,"Dear local newspaper, I think effects computer...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,NaN,9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,NaN,7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,NaN,10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,"Dear @LOCATION1, I know having computers has a...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
df.tail(5)

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
12971,21626,8,In most stories mothers and daughters are eit...,17,18,NaN,35,NaN,NaN,NaN,...,4.0,4.0,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
12972,21628,8,I never understood the meaning laughter is th...,15,17,NaN,32,NaN,NaN,NaN,...,4.0,4.0,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
12973,21629,8,"When you laugh, is @CAPS5 out of habit, or is ...",20,26,40.0,40,NaN,NaN,NaN,...,5.0,5.0,5.0,5.0,4.0,4.0,4.0,4.0,4.0,4.0
12974,21630,8,Trippin' on fen...,20,20,NaN,40,NaN,NaN,NaN,...,4.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
12975,21633,8,Many people believe that laughter can improve...,20,20,NaN,40,NaN,NaN,NaN,...,4.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
# Getting more info about the test info

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12976 entries, 0 to 12975
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   essay_id        12976 non-null  int64  
 1   essay_set       12976 non-null  int64  
 2   essay           12976 non-null  object 
 3   rater1_domain1  12976 non-null  int64  
 4   rater2_domain1  12976 non-null  int64  
 5   rater3_domain1  128 non-null    float64
 6   domain1_score   12976 non-null  int64  
 7   rater1_domain2  1800 non-null   float64
 8   rater2_domain2  1800 non-null   float64
 9   domain2_score   1800 non-null   float64
 10  rater1_trait1   2292 non-null   float64
 11  rater1_trait2   2292 non-null   float64
 12  rater1_trait3   2292 non-null   float64
 13  rater1_trait4   2292 non-null   float64
 14  rater1_trait5   723 non-null    float64
 15  rater1_trait6   723 non-null    float64
 16  rater2_trait1   2292 non-null   float64
 17  rater2_trait2   2292 non-null  

In [63]:
df.shape


(12976, 28)

In [64]:
# Let's check how many values are none in each row of the train dataset
df.isnull().sum()

essay_id              0
essay_set             0
essay                 0
rater1_domain1        0
rater2_domain1        0
rater3_domain1    12848
domain1_score         0
rater1_domain2    11176
rater2_domain2    11176
domain2_score     11176
rater1_trait1     10684
rater1_trait2     10684
rater1_trait3     10684
rater1_trait4     10684
rater1_trait5     12253
rater1_trait6     12253
rater2_trait1     10684
rater2_trait2     10684
rater2_trait3     10684
rater2_trait4     10684
rater2_trait5     12253
rater2_trait6     12253
rater3_trait1     12848
rater3_trait2     12848
rater3_trait3     12848
rater3_trait4     12848
rater3_trait5     12848
rater3_trait6     12848
dtype: int64

In [65]:
# Since a large part of the dataset has columns with more than 70-80 percent missing values so Deleting those columns
df.dropna(axis = 1, inplace = True)
df.head()

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",4,4,8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,10
4,5,1,"Dear @LOCATION1, I know having computers has a...",4,4,8


In [66]:
df = df.rename(columns={'essay': 'text', 'domain1_score': 'score'})
df

,essay_id,essay_set,text,rater1_domain1,rater2_domain1,score
0,1,1,"Dear local newspaper, I think effects computer...",4,4,8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,10
4,5,1,"Dear @LOCATION1, I know having computers has a...",4,4,8
...,...,...,...,...,...,...
12971,21626,8,In most stories mothers and daughters are eit...,17,18,35
12972,21628,8,I never understood the meaning laughter is th...,15,17,32
12973,21629,8,"When you laugh, is @CAPS5 out of habit, or is ...",20,26,40
12974,21630,8,Trippin' on fen...,20,20,40


In [67]:
# Describing the train set
df.describe()

,essay_id,essay_set,rater1_domain1,rater2_domain1,score
count,12976.000000,12976.000000,12976.000000,12976.000000,12976.000000
mean,10295.395808,4.179485,4.127158,4.137408,6.800247
std,6309.074105,2.136913,4.212544,4.264330,8.970705
min,1.000000,1.000000,0.000000,0.000000,0.000000
25%,4438.750000,2.000000,2.000000,2.000000,2.000000
50%,10044.500000,4.000000,3.000000,3.000000,3.000000
75%,15681.250000,6.000000,4.000000,4.000000,8.000000
max,21633.000000,8.000000,30.000000,30.000000,60.000000


In [68]:
# Checking how many unique essay id were given
print(df['essay_set'].nunique())
df['essay_set'].unique()

8


array([1, 2, 3, 4, 5, 6, 7, 8])

In [69]:
# Counting the number of each essay_set

print(df.groupby('essay_set').size())

essay_set
1    1783
2    1800
3    1726
4    1770
5    1805
6    1800
7    1569
8     723
dtype: int64


In [70]:
def remove_stop_words(dataframe, target_column_name, new_column_name):
    dataframe[new_column_name] = dataframe[target_column_name].apply(lambda x: ''.join([item for item in x.split() if item not in stopwords.words('english')]))
    return dataframe

def remove_punctuations(dataframe, target_column_name, new_column_name):
    dataframe[new_column_name] = dataframe[target_column_name].apply(lambda x: "".join([char for char in x if char not in string.punctuation]))
    return dataframe

In [71]:
  import nltk
  nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [72]:
cl_df = remove_stop_words(df, 'text', 'cl_essay')

In [73]:
cl_df.head

<bound method NDFrame.head of        essay_id  essay_set                                               text  \
0             1          1  Dear local newspaper, I think effects computer...   
1             2          1  Dear @CAPS1 @CAPS2, I believe that using compu...   
2             3          1  Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...   
3             4          1  Dear Local Newspaper, @CAPS1 I have found that...   
4             5          1  Dear @LOCATION1, I know having computers has a...   
...         ...        ...                                                ...   
12971     21626          8   In most stories mothers and daughters are eit...   
12972     21628          8   I never understood the meaning laughter is th...   
12973     21629          8  When you laugh, is @CAPS5 out of habit, or is ...   
12974     21630          8                                 Trippin' on fen...   
12975     21633          8   Many people believe that laughter can improve...  

In [74]:
cl_df = remove_punctuations(df, 'text', 'cl_essay')

In [75]:
cl_df.head

<bound method NDFrame.head of        essay_id  essay_set                                               text  \
0             1          1  Dear local newspaper, I think effects computer...   
1             2          1  Dear @CAPS1 @CAPS2, I believe that using compu...   
2             3          1  Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...   
3             4          1  Dear Local Newspaper, @CAPS1 I have found that...   
4             5          1  Dear @LOCATION1, I know having computers has a...   
...         ...        ...                                                ...   
12971     21626          8   In most stories mothers and daughters are eit...   
12972     21628          8   I never understood the meaning laughter is th...   
12973     21629          8  When you laugh, is @CAPS5 out of habit, or is ...   
12974     21630          8                                 Trippin' on fen...   
12975     21633          8   Many people believe that laughter can improve...  

In [76]:
cl_tr = cl_df.copy()

In [77]:
cl_tr.head

<bound method NDFrame.head of        essay_id  essay_set                                               text  \
0             1          1  Dear local newspaper, I think effects computer...   
1             2          1  Dear @CAPS1 @CAPS2, I believe that using compu...   
2             3          1  Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...   
3             4          1  Dear Local Newspaper, @CAPS1 I have found that...   
4             5          1  Dear @LOCATION1, I know having computers has a...   
...         ...        ...                                                ...   
12971     21626          8   In most stories mothers and daughters are eit...   
12972     21628          8   I never understood the meaning laughter is th...   
12973     21629          8  When you laugh, is @CAPS5 out of habit, or is ...   
12974     21630          8                                 Trippin' on fen...   
12975     21633          8   Many people believe that laughter can improve...  

In [78]:
#remove numbers from strings in team column
cl_tr['cl_essay'] = cl_tr['cl_essay'].str.replace('\d+', '', regex=True)

In [79]:
#remove 'CAPS' from strings in team column
cl_tr['cl_essay'] = cl_tr['cl_essay'].str.replace('CAPS', '')

In [80]:
cl_tr.head()

,essay_id,essay_set,text,rater1_domain1,rater2_domain1,score,cl_essay
0,1,1,"Dear local newspaper, I think effects computer...",4,4,8,Dear local newspaper I think effects computers...
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,9,Dear I believe that using computers will ben...
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,7,Dear More and more people use computers but...
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,10,Dear Local Newspaper I have found that many e...
4,5,1,"Dear @LOCATION1, I know having computers has a...",4,4,8,Dear LOCATION I know having computers has a po...


In [81]:
#remove 'LOCATION' from strings in team column
cl_tr['cl_essay'] = cl_tr['cl_essay'].str.replace('LOCATION', '')
cl_tr.head()

,essay_id,essay_set,text,rater1_domain1,rater2_domain1,score,cl_essay
0,1,1,"Dear local newspaper, I think effects computer...",4,4,8,Dear local newspaper I think effects computers...
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,9,Dear I believe that using computers will ben...
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,7,Dear More and more people use computers but...
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,10,Dear Local Newspaper I have found that many e...
4,5,1,"Dear @LOCATION1, I know having computers has a...",4,4,8,Dear I know having computers has a positive e...


In [82]:
print(cl_tr['cl_essay'])

0        Dear local newspaper I think effects computers...
1        Dear   I believe that using computers will ben...
2        Dear    More and more people use computers but...
3        Dear Local Newspaper  I have found that many e...
4        Dear  I know having computers has a positive e...
                               ...                        
12971     In most stories mothers and daughters are eit...
12972     I never understood the meaning laughter is th...
12973    When you laugh is  out of habit or is  cause W...
12974                                   Trippin on fenc...
12975     Many people believe that laughter can improve...
Name: cl_essay, Length: 12976, dtype: object


In [83]:
cl_tr = cl_tr.loc[df['score']<21]

In [84]:
cl_tr

,essay_id,essay_set,text,rater1_domain1,rater2_domain1,score,cl_essay
0,1,1,"Dear local newspaper, I think effects computer...",4,4,8,Dear local newspaper I think effects computers...
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,9,Dear I believe that using computers will ben...
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,7,Dear More and more people use computers but...
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,10,Dear Local Newspaper I have found that many e...
4,5,1,"Dear @LOCATION1, I know having computers has a...",4,4,8,Dear I know having computers has a positive e...
...,...,...,...,...,...,...,...
12375,20871,8,Laughter is good for every one.@CAPS1 helps th...,10,15,20,Laughter is good for every one helps the body ...
12444,20954,8,DUE you think that being part of sameting f...,10,5,15,DUE you think that being part of sameting f...
12455,20967,8,I was raised with two brother that mean the wo...,10,10,20,I was raised with two brother that mean the wo...
12789,21389,8,"Its said that laughing is a good part of life,...",10,10,20,Its said that laughing is a good part of life ...


In [85]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [86]:
import torch
import numpy as np
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {0:0,
          1:1,
          2:2,
          3:3,
          4:4,
          5:5,
          6:6,
          7:7,
          8:8,
          9:9,
          10:10,
          11:11,
          12:12,
          13:13,
          14:14,
          15:15,
          16:16,
          17:17,
          18:18,
          19:19,
          20:20
          }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in cl_tr['score']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in cl_tr['cl_essay']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [87]:
np.random.seed(112)
df_train, df_val, df_test = np.split(cl_tr.sample(frac=1, random_state=42), 
                                     [int(.8*len(cl_tr)), int(.9*len(cl_tr))])

print(len(df_train),len(df_val), len(df_test))

9584 1198 1198


In [88]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 21)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [89]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  
EPOCHS = 5
model = BertClassifier()
LR = 1e-6
              
train(model, df_train, df_val, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 5990/5990 [05:42<00:00, 17.50it/s]


Epochs: 1 | Train Loss:  1.156                 | Train Accuracy:  0.538                 | Val Loss:  6.976                 | Val Accuracy:  5.533


100%|██████████| 5990/5990 [05:39<00:00, 17.66it/s]


Epochs: 2 | Train Loss:  0.816                 | Train Accuracy:  0.704                 | Val Loss:  6.135                 | Val Accuracy:  5.833


100%|██████████| 5990/5990 [05:39<00:00, 17.62it/s]


Epochs: 3 | Train Loss:  0.729                 | Train Accuracy:  0.750                 | Val Loss:  5.285                 | Val Accuracy:  6.374


100%|██████████| 5990/5990 [05:39<00:00, 17.63it/s]


Epochs: 4 | Train Loss:  0.659                 | Train Accuracy:  0.792                 | Val Loss:  4.875                 | Val Accuracy:  6.684


100%|██████████| 5990/5990 [05:39<00:00, 17.65it/s]


Epochs: 5 | Train Loss:  0.604                 | Train Accuracy:  0.835                 | Val Loss:  4.465                 | Val Accuracy:  7.013
